# HOMEWORK 2: NUMPY FOR DATA SCIENCE

Họ tên: Lê Hà Thanh Chương

MSSV: 23120195

---

## Import các thư viện cần thiết

In [1]:
import numpy as np
import sys
import os
sys.path.append('../src')
from data_processing import process_hr_data, impute_missing, prepare_matrices, export_processed_data

---

## Tiền xử lý

Dựa trên những phát hiện quan trọng thu được ở bước khám phá dữ liệu (EDA) — đặc biệt là việc toàn bộ các biến được đọc dưới dạng chuỗi và sự tồn tại của nhiều biến ordinal — giai đoạn tiền xử lý được triển khai nhằm chuẩn hóa cấu trúc dữ liệu, đảm bảo mỗi biến phản ánh đúng bản chất định tính hoặc định lượng của nó, đồng thời thiết lập một cấu trúc dữ liệu nhất quán và tạo nền tảng cho các bước phân tích và mô hình hóa tiếp theo.

### Load raw data for preprocessing

Trước hết, dữ liệu thô được tải lại từ tệp CSV và chuyển sang NumPy array để thuận tiện cho các bước tiền xử lý tiếp theo. Lưu ý rằng, dữ liệu đã được khám phá trong notebook 01_data_exploration, do đó bước này chỉ tập trung vào việc chuẩn bị dữ liệu cho pipeline preprocessing, bao gồm các thao tác chuẩn hóa kiểu dữ liệu, xử lý missing values và mapping ordinal

In [2]:
DATA_PATH = "../data/raw/aug_train.csv"

# Load the dataset
raw_data = np.genfromtxt(DATA_PATH, delimiter = ",", dtype = str, encoding = "utf-8")

print("Raw data loaded successfully!")
print(f"Shape: {raw_data.shape}")
print("Header:", raw_data[0])
print("First row sample:", raw_data[1])

Raw data loaded successfully!
Shape: (19159, 14)
Header: ['enrollee_id' 'city' 'city_development_index' 'gender'
 'relevent_experience' 'enrolled_university' 'education_level'
 'major_discipline' 'experience' 'company_size' 'company_type'
 'last_new_job' 'training_hours' 'target']
First row sample: ['8949' 'city_103' '0.92' 'Male' 'Has relevent experience' 'no_enrollment'
 'Graduate' 'STEM' '>20' '' '' '1' '36' '1.0']


---

### Data Type Correction & Initial Preprocessing

Sau khi dữ liệu thô được tải và chuyển sang dạng structured array (`raw_data`), bước tiếp theo là chuẩn hóa kiểu dữ liệu và thực hiện các thao tác preprocessing ban đầu nhằm đảm bảo tính nhất quán và phù hợp với các bước phân tích sau.


#### (A) Biến số (Numerical features)

Hai biến mang bản chất liên tục được xử lý như sau:

* `city_development_index` được chuyển sang kiểu `float`, phản ánh mức độ phát triển của thành phố nơi ứng viên cư trú.
* `training_hours` cũng được lưu dưới dạng `float` để thuận tiện cho việc xử lý các giá trị missing, mặc dù bản chất dữ liệu là số nguyên.

Việc giữ dạng float giúp đảm bảo khả năng áp dụng các phương pháp tính toán thống kê và scaling mà không làm mất thông tin do missing values.

#### (B) Biến thứ tự (Ordinal features)

Các biến ordinal trong dataset phản ánh quan hệ thứ bậc và do đó không thể xử lý như biến phân loại thông thường. Chúng được ánh xạ sang thang số nguyên theo các quy tắc nhất quán:

* `experience`: các giá trị từ "<1" đến ">20" được ánh xạ vào dãy số nguyên từ 0 đến 21.
* `company_size`: chuẩn hóa các biểu diễn như "10/49" -> "10-49", sau đó mapping vào thang số nguyên phản ánh quy mô doanh nghiệp.
* `last_new_job`: 'never', '1', …, '>4' được chuyển thành số nguyên thứ tự tương ứng từ 0 đến 5, phản ánh số năm kể từ công việc mới nhất.
* `education_level`: các mức trình độ từ 'Primary School', 'High School', …, 'Phd' được mã hóa thành thang điểm thứ bậc 0 – 4.

Các giá trị trống hoặc chuỗi rỗng trong các cột numerical và ordinal được chuyển thành `np.nan` để thuận tiện cho các bước xử lý missing values tiếp theo.

#### (C) Biến phân loại (Categorical nominal features)

Nhóm các biến phân loại bao gồm:

* `gender`, `relevent_experience`, `enrolled_university`, `major_discipline`, `company_type`, `city`

Chúng được xử lý theo hai cơ chế khác nhau:

1. Frequency Encoding cho biến `city`: Thay vì giữ nguyên dạng chuỗi, biến `city` được ánh xạ thành frequency encoding. Mỗi giá trị thành phố được thay bằng tần suất xuất hiện của nó trong dữ liệu, giúp giữ thông tin phân bố mà không tạo ra ma trận quá lớn khi one-hot encode làm giảm đáng kể số chiều của dữ liệu.

2. One-hot Encoding cho các biến còn lại (`gender`, `relevent_experience`, `enrolled_university`, `major_discipline`, `company_type`)
   * Giá trị trống (`''`) được thay bằng `'Unknown'`.
   * Sau đó, áp dụng one-hot encoding để chuyển thành ma trận nhị phân, đảm bảo tính tương thích với các thuật toán học máy tuyến tính và phi tuyến.

#### Kết quả tiền xử lý

Ví dụ kiểm tra sau khi mapping cho thấy tính chính xác của quá trình chuyển đổi:

* `'>20' -> 21`, `'<1' -> 0` trong biến `experience`
* các nhóm `company_size` được chuẩn hóa và mã hóa thành số nguyên
* tần suất `city` được tính chính xác theo phân bố quan sát

In [3]:
processed_data = process_hr_data(raw_data)
print("Preprocessing complete.")
print("Keys available:", processed_data.keys())

# Show some samples
print(f"\nExperience (Before mapping): {raw_data[1:, 8][:5]}")
print(f"Experience (After mapping):  {processed_data['experience'][:5]}")
print(f"Company Size (After mapping): {processed_data['company_size'][:5]}")

Preprocessing complete.
Keys available: dict_keys(['city_development_index', 'training_hours', 'experience', 'company_size', 'last_new_job', 'education_level', 'city_freq', 'gender_onehot', 'relevent_experience_onehot', 'enrolled_university_onehot', 'major_discipline_onehot', 'company_type_onehot', 'target'])

Experience (Before mapping): ['>20' '15' '5' '<1' '>20']
Experience (After mapping):  [21. 15.  5.  0. 21.]
Company Size (After mapping): [nan  2. nan nan  2.]


---

### Missing Value Imputation

Sau khi hoàn tất bước chuẩn hóa kiểu dữ liệu cho các biến numerical và ordinal, một số trường vẫn còn xuất hiện giá trị thiếu (*missing values*, biểu diễn dưới dạng `np.nan`). Việc xử lý missing values là yêu cầu thiết yếu nhằm đảm bảo tính toàn vẹn của dữ liệu, đồng thời duy trì khả năng áp dụng các phương pháp thống kê và mô hình học máy vốn không chấp nhận các giá trị trống.

#### Chiến lược xử lý missing values

Numerical và ordinal features (`city_development_index`, `training_hours`, `experience`, `company_size`): Các biến định lượng và thứ tự được xử lý bằng phương pháp thay thế giá trị thiếu bằng trung vị (median) của cột tương ứng. Phương pháp này giữ nguyên phân bố dữ liệu và hạn chế ảnh hưởng của các giá trị ngoại lai.

Categorical nominal features: Các giá trị missing được giữ nguyên ở giai đoạn này; sẽ được xử lý sau nếu cần (ví dụ, khi áp dụng one-hot encoding hoặc target encoding).



In [4]:
imputed_data = impute_missing(processed_data)

# Check again if there are any NaNs
cols_check = ['city_development_index', 'training_hours', 'experience', 'company_size', 'last_new_job', 'education_level']
for col in cols_check:
    n_missing = np.sum(np.isnan(imputed_data[col]))
    print(f"{col} - missing values: {n_missing}")

city_development_index - missing values: 0
training_hours - missing values: 0
experience - missing values: 0
company_size - missing values: 0
last_new_job - missing values: 0
education_level - missing values: 0


Quá trình xử lý được kiểm tra bằng cách đếm lại số lượng `np.nan` trong những cột numerical và ordinal chủ chốt. Kết quả cho thấy toàn bộ các feature số đã được khử missing hoàn toàn, đảm bảo sẵn sàng cho các bước tiếp theo như chuẩn hóa (scaling), tạo ma trận đặc trưng và tách tập train/test.

---

### Feature Matrix Construction

Sau khi dữ liệu đã được chuẩn hóa kiểu dữ liệu và xử lý đầy đủ các giá trị khuyết, bước tiếp theo trong quy trình tiền xử lý là xây dựng ma trận đặc trưng (*feature matrix*) phục vụ cho các thuật toán học máy.
Mục tiêu của bước này là kết hợp có hệ thống toàn bộ các nhóm biến, bao gồm numerical, ordinal, frequency-encoded và one-hot encoded thành một cấu trúc dữ liệu duy nhất ($X$), nhất quán và tách biệt vector nhãn mục tiêu ($y$).

Việc xây dựng ma trận đặc trưng được thực hiện thông qua hàm `prepare_matrices()` trong module `src/data_processing.py`.

#### Cơ chế hoạt động của hàm

* Nhóm biến định lượng và thứ tự: Các biến như `city_development_index`, `training_hours`, `experience`, `company_size`, `last_new_job`, `education_level`, `city_freq`(frequency encoding) được ghép cột (`np.column_stack`) để tạo thành ma trận đặc trưng dạng số.
* Nhóm biến One-hot: Tự động thu thập tất cả các ma trận con có hậu tố _onehot (ví dụ: `gender_onehot`, `major_discipline_onehot`). Sau đó toàn bộ ma trận này được ghép theo chiều cột.
* Hợp nhất: Ghép toàn bộ thành ma trận $X$ có kích thước $N \times M$, trong đó $N$ là số mẫu và $M$ là tổng số chiều đặc trưng. Kích thước thu được: n_samples × (7 + tổng số chiều one-hot)
* Tách nhãn: Vector $y$ (`target`) được tách riêng phục vụ cho việc huấn luyện có giám sát.

In [5]:
X, y = prepare_matrices(imputed_data)

print("Feature matrix construction complete!")
print("X shape:", X.shape)
print("y shape:", y.shape)

Feature matrix construction complete!
X shape: (19158, 31)
y shape: (19158,)


In [6]:
unique, counts = np.unique(y, return_counts=True) 
print(f"Class distribution: {dict(zip(unique, counts))}")

Class distribution: {0: 14381, 1: 4777}


---

### Export Clean Dataset

Sau khi hoàn tất tiền xử lý, chúng ta xuất dữ liệu ra hai định dạng:

* `train_preprocessed.npy`: Dùng cho Notebook 03 (Modeling). Định dạng nhị phân của NumPy giúp load dữ liệu nhanh và giữ nguyên kiểu dữ liệu (không bị lỗi parsing).

* `train_preprocessed.csv`: Dùng cho mục đích kiểm tra, báo cáo và bảo đảm tính tương thích với các notebook.

Logic tạo header và xuất file được đóng gói trong hàm export_processed_data.

In [7]:
export_processed_data(X = X, y = y, raw_data = raw_data, imputed_data = imputed_data, output_dir="../data/processed")

Header columns: 32
Matrix columns: 32
CSV exported to: ../data/processed/train_preprocessed.csv
NPY files exported to: ../data/processed
